# Lab 06: Modeling leaf and canopy photosynthesis

<div class="alert alert-warning">
If figures/tables are not rendered correctly, make sure your notebook is `trusted` by choosing `Trust Notebook` command found in `View` - `Activate Command Palette` menu. Typing in the first letters (i.e. "tr") should give you the command.
</div>

In [ ]:
using Cropbox

## Ex 6.1
Using Cropbox package, create a function that implements the Michaelis-Mention equation as shown in Eqn[6.3] that calculates the rate of biochemical reaction by an enzyme. Use the parameter values provided in Table 6.1. Plot the response of reaction rate ($V$) as a function of substrate concentration ranging from 0 to 1000. For this exercise we will ignore units but imagine that the substrate is CO2 used for carboxylation catalyzed by the enzyme Rubisco.

- Eqn 6.3

$$
\frac{d[P]}{dt} = V = \frac{V_{\mathrm{max}} \cdot [S]}{K_m + [S]}
$$

- Table 6.1

| Symbol | Value | Units | Description |
|:-------|:------|:------|:------------|
| $V_{\mathrm{max}}$ | 100 | [Product] / time (*e.g.* $\mathrm{\mu mol}\ \mathrm{m^{-2}}\ \mathrm{s^{-1}}$) | Potential rate of biochemical reaction for product synthesis (*i.e.* Rubisco capacity) |
| $K_m$ | 400 | [Substrate] (*e.g.* $\mathrm{M}$, $\mathrm{ppm}$, $\mathrm{\mu mol}\ \mathrm{mol^{-1}}$) | Michaelis-Menten constant where $V = \frac{1}{2} V_{\mathrm{max}}$ |

In [ ]:
@system MichaelisMenten(Controller) begin
    V(S, Vmax, Km): reaction_rate => begin
        Vmax * S / (Km + S)
    end ~ track(u"μmol/m^2/s")

    Vmax: maximum_reaction_rate     => 100 ~ preserve(parameter, u"μmol/m^2/s")
    Km:   michaelis_menten_constant => 400 ~ preserve(parameter, u"μmol/mol")

    S:    substrate                 => 0   ~ preserve(parameter, u"μmol/mol")
end

In [ ]:
instance(MichaelisMenten)

### Simulation with configuration

By default, `simulate()` runs a single simulation for a given configuration set from `config` option. If we want to run multiple simulations, a list of multiple configurations should be provided in `configs` option (note trailing `s`). Cropbox provides a helper macro `@config` to easily generate such a list of multiple configurations useful for some common use cases such as adjusting one parameter over a range of values. Here we want `S` value ranging from 0 to 1000 by 1 (`0:1:1000` or `0:1000` as the step size is 1 by default) which is declared as a Julia built-in range type and later expanded to a list by `!` operator used in conjunction with the `@config` macro. By the way, the name of system `:0` in the configuration is a shorthand for the name of whatever top-level system currently used for simulation. In this example, it is going to be `MichaelisMenten`.

In [ ]:
mmc = @config !(:0 => :S => 0:1000);

In [ ]:
r = simulate(MichaelisMenten; configs = mmc);

Note that `time` doesn't advance here indicating that this is a `steady-state` simulation

In [ ]:
visualize(r, :S, :V; kind = :line)

We can use another form of `visualize()` for automatically running `simulate()` inside. A good thing about this approach is that we don't have to deal with custom expansion of configurations using `@config` in favor of convenient options like `xstep`. `xstep` indicates the range of a certain parameter value that needs to be simulated individually to make a series of consecutive output. Here we change the value of `S` from 0 to 1000 with an increment of 10 by simply providing a snippet of range value (`:0 => :S => 0:10:1000`) representing 101 times of simulation. Common plotting options like `kind` are also available.

In [ ]:
visualize(MichaelisMenten, :S, :V; 
    xstep = :0 => :S => 0:10:1000,
    kind = :line,
)

Create a configuration to change `Km` and `Vmax`, and observe how `V` changes with different`Km` and `Vmax` values over a wide range of `S` levels

In [ ]:
c1 = @config(
    :MichaelisMenten => (
        Km = 200,
        Vmax = 50,
        ),
    )

Run the simulation with new configuation `c1` over a range of `S` specified in `mmc`.

In [ ]:
r = simulate(MichaelisMenten; configs = mmc, config = c1);

We can simply `visualize` this using `xstep` argument with new configuration `c1`.

In [ ]:
visualize(MichaelisMenten, :S, :V; 
    config = c1,
    xstep = :0 => :S => 0:10:2000,
    kind = :line,
)

## Ex 6.2

In this exercise, you will answer Q 6.5 using Julia (or R or Python or any computer tool of your choice).
- Get an analytical solution for Eqn[5.4]
- Create a function to get numerical solutions using Gaussian two-point and three-point integration methods.

- Eqn 6.4

$$
\int_0^{\pi} \sin{x} \, dx
$$

### Analytical solution

In [ ]:
f′(x) = sin(x)
f(x) = -cos(x)
f(π) - f(0)

### Gaussian two-point integration method

As the two points (-1/√3, +1/√3) and weights (1 for both) are defined in a normalized interval [-1, 1], we need to shift and scale them to the interval that our problem lies on, [0, π]. This process is shown step by step below.

##### [-1, 1]

In [ ]:
(1 * f′(-1/√3) + 1 * f′(1/√3)) * 2/2

##### [0, 2]

In [ ]:
(1 * f′(-1/√3 + 1) + 1 * f′(1/√3 + 1)) * 2/2

##### [0, 1]

In [ ]:
(1 * f′((-1/√3 + 1) / 2) + 1 * f′((1/√3 + 1) / 2)) * 1/2

##### [0, π]

In [ ]:
(1 * f′((-1/√3 + 1) / 2 * π) + 1 * f′((1/√3 + 1) / 2 * π)) * π/2

#### Gaussian three-point integration method

In [ ]:
(5/9 * f′((-√(3/5) + 1) / 2 * π) + 8/9 * f′((0 + 1)/2 * π) + 5/9 * f′((√(3/5) + 1) / 2 * π)) * π/2

Calculation of Gaussian quadrature can be more easily done via a dedicated package called [QuadGK.jl](https://github.com/JuliaMath/QuadGK.jl) which is already imported inside Cropbox.

In [ ]:
X, W = Cropbox.QuadGK.gauss(2, 0, pi)
sum(W .* f′.(X))

In [ ]:
X, W = Cropbox.QuadGK.gauss(3, 0, pi)
sum(W .* f′.(X))

## Ex 6.3

Using Cropbox package, create a function to predict leaf net photosynthesis ($A$) as a function of irradiance ($I$) that takes parameters of photochemical efficiency ($\alpha$), the maximum photosynthetic rate at saturating light ($A_{\mathrm{max}}$), and dark respiration rate ($R_d$).
- Based on rectangular hyperbola (Eqn[5.11])
- Based on non-rectangular hyperbola (Eqn[5.12] for $0 < \theta \leq 1$)
- Using the parameter values in 6.1, plot the light response curves of both functions from 0 to 2000 $\mathrm{\mu mol}\ \mathrm{m^{-2}}\ \mathrm{s^{-1}}$ of photosynthetic photon flux density (PFD).
- Evaluate the sensitivity of non-rectangular hyperbola to $\theta$ graphically and discuss.

### Rectangular hyperbola

- Eqn 6.11

$$
\begin{align}
A_g &= \frac{\alpha I A_{\mathrm{max}}}{\alpha I + A_{\mathrm{max}}} \\
A_n &= A_g - R_d
\end{align}
$$

- Table 6.1

| Symbol | Value | Units | Description |
|:-------|:------|:------|:------------|
| $\alpha$ | 0.05 | $$\mathrm{\mu mol_{CO_2}}\ \mathrm{\mu mol_{photon}^{-1}}$$ | Apparent photochemical efficiency (*a.k.a.* quantum yield) |
| $A_{\mathrm{max}}$ | 25.0 | $\mathrm{\mu mol}\ \mathrm{m^{-2}}\ \mathrm{s^{-1}}$ | Light saturated $A_g$ at ambient $\mathrm{[CO_2]}$ and standard temperature |
| $R_d$ | 1.0, 40 | $\mathrm{\mu mol_{CO_2}}\ \mathrm{m^{-2}}\ \mathrm{s^{-1}}$ | Leaf mitochondrial respiration rate at 25 $\mathrm{^{\circ}C}$ |

- Table 6.1

| Symbol | Units | Description |
|:-------|:------|:------------|
| $I$ | $\mathrm{\mu mol_{photon}}\ \mathrm{m^{-2}}$ | Incident PAR on leaf surface |
| $A_g$ | $$\mathrm{\mu mol_{CO_2}}\ \mathrm{m^{-2}}\ \mathrm{s^{-1}}$$ | Leaf gross $\mathrm{CO_2}$ assimilation rate 
| $A_n$ | $\mathrm{\mu mol_{CO_2}}\ \mathrm{m^{-2}}\ \mathrm{s^{-1}}$ | Leaf net $\mathrm{CO_2}$ assimilation rate |

We first define a mix-in system named `Photosynthesis` which provides a shared template of photosynthesis model which other models will be all based on.

We have three common parameters and two variables representing photosynthesis rate. $A_n$ is the net photosynthesis derived from gross photosynthesis ($A_g$) and respiration ($R_d$). As we decided to make $R_d$ a constant parameter, subsequent models would only need to provide a custom definition for $A_g$. Although `Ag` is not defined here, it is still a dependency of `An` and thus we need to make a placeholder for it by creating a `hold` variable. For declaring `hold`, only a name and an optinal alias of the variable are needed. Any variables with `hold` state should be properly defined later in another system which embeds `Photosynthesis` as a mix-in.

In [ ]:
@system Photosynthesis begin
    α:    photochemical_efficiency    => 0.05 ~ preserve(parameter)
    Amax: maximum_photosynthetic_rate => 25   ~ preserve(parameter, u"μmol/m^2/s")
    Rd:   dark_respiration            => 1.0  ~ preserve(parameter, u"μmol/m^2/s")

    Ag: gross_photosynthesis ~ hold

    An(Ag, Rd): net_photosynthesis => Ag - Rd ~ track(u"μmol/m^2/s")
end

Here is our first model based on rectangular hyperbola named `PhotosynthesisRH` which consists of two mix-ins, `Photosynthesis` and `Controller`. As you remember, `Controller` is a mix-in that makes a top-level system which can be instantiated by `instance()` or `simulate()`. Note that variables from other mix-in that are used by the system should be declared as `hold` kind of variable to ensure correct bindings between variables. Note that `PhotosynthesisRH` now has a concrete definition of `Ag` with an introduction of additional parameter `I`.

In [ ]:
@system PhotosynthesisRH(Photosynthesis, Controller) begin
    I: irradiance => 0 ~ preserve(parameter, u"μmol/m^2/s")

    Ag(I, α, Amax): gross_photosynthesis => begin
        α*I * Amax / (α*I + Amax)
    end ~ track(u"μmol/m^2/s")
end

We use `visualize()` with a range of `I` value set up for `xstep` in the same way as we saw earlier.

In [ ]:
visualize(PhotosynthesisRH, :I, :An;
    xstep = :0 => :I => 0:1500,
    kind = :line,
)

### Non-rectangular hyperbola

- Eqn 6.12

$$
A_g = \frac{\alpha I + A_{\mathrm{max}} - \sqrt{(\alpha I + A_{\mathrm{max}})^2 - 4\theta\alpha I A_{\mathrm{max}}}}{2\theta} \\
$$

- Table 6.1

| Symbol | Value | Units | Description |
|:-------|:------|:------|:------------|
| $\theta$ | 0.7 | - | Curvature determining transition between light-limited and saturated photosynthesis |

Another variant of the model using non-rectangular hyperbola looks similar to the rectangular model we built earlier, except the addition of new parameter `θ` controlling curvature of transition.

In [ ]:
@system PhotosynthesisNRH(Photosynthesis, Controller) begin
    θ: transition_curvature => 0.7 ~ preserve(parameter)
    I: irradiance           => 0   ~ preserve(parameter, u"μmol/m^2/s")

    Ag(θ, I, α, Amax): gross_photosynthesis => begin
        (α*I + Amax - √((α*I + Amax)^2 - 4θ*α*I*Amax)) / 2θ
    end ~ track(u"μmol/m^2/s")
end

In [ ]:
visualize(PhotosynthesisNRH, :I, :An;
    xstep = :0 => :I => 0:1500,
    kind = :line,
)

By the way, instead of solving the quadratic equation for $A_g$ manually as we did above, we can use `solve` variable to let the framework automatically figure it out. In such case, we can use Eqn[5.9] almost as is. When multiple solutions exist, we may need to provide an extra information to guide a solution. Here we specify a sensible range of $A_g$ from 0 to $A_{\mathrm{max}}$ with `lower` and `upper` tags.

- Eqn 6.9.

$$
\theta A_g^2 - (\alpha I + A_{\mathrm{max}}) A_g + \alpha I A_{\mathrm{max}} = 0
$$

In [ ]:
@system PhotosynthesisNRH2(Photosynthesis, Controller) begin
    θ: transition_curvature => 0.7 ~ preserve(parameter)
    I: irradiance           => 0   ~ preserve(parameter, u"μmol/m^2/s")

    Ag(Ag, θ, I, α, Amax): gross_photosynthesis => begin
        θ*Ag^2 - (α*I + Amax)*Ag + α*I*Amax
    end ~ solve(lower = 0, upper = Amax, u"μmol/m^2/s")
end

In [ ]:
visualize(PhotosynthesisNRH2, :I, :An;
    xstep = :0 => :I => 0:1500,
    kind = :line,
)

#### Sensitivity of θ

So far we've only made plots with a single line composed of a series of points from multiple simulations triggered by `xstep` option. For drawing multiple lines of simulations for non-rectangular hyperbola, we use `group` option to control another parameter value in the system. For testing sensitivity of `θ`, we want to run simulations with different values of `θ` from 0.2 to 1.0 by increment of 0.2. For convenience of the reader, we use a parameter list in reverse order (`[1, 0.99, 0.9, 0.7, 0.4]`) to show the labels from top to bottom. Additionally, we append a plot for the rectangular hyperbola obtained when `θ` is 0.

In [ ]:
let x = :I, y = :An,
    xstep = :0 => :I => 0:10:1500,
    group = :0 => :θ => [1, 0.99, 0.9, 0.7, 0.4],
    kind = :line
    p = visualize(PhotosynthesisNRH, x, y; group, xstep, kind)
    visualize!(p, PhotosynthesisRH, x, y; xstep, kind, name = "0")
end

## Ex 6.4

Using Cropbox, you will evaluate three different systems for modeling instantaneous canopy photosynthesis ($A_{g,\mathrm{can}}$) based on the Appendix (section A.2) of Goudriaan and van Laar (1994) and also Goudriaan (1986):
- Build system based on 1) analytical solution, 2) Gaussian 3 point integration method, and 3) Rectangular integration (Eulerian) method over total LAI ($L$) with an increment ($dl$) = 1.0 and 0.1.
- Reproduce the results shown in  Appendix (section A.2) of Goudriaan and van Laar(1994) for the same conditions
- Plot *gross* canopy photosynthesis ($A_{g,\mathrm{can}}$) to compare the three approaches in response to LAI ranging from 0 to 10
- Plot *net* canopy photosynthesis ($A_{n,\mathrm{can}}$) to compare the three approaches in response to LAI ranging from 0 to 10. Assume that canopy respiration is 10% of $A_{max}$ integrated over the whole canopy. How do the two plots compare?
- Choose one method of integration and experiment with different parameter values for physiological (e.g., $A_{\mathrm{max}}$, $\alpha$, $\theta$) and canopy (e.g., $k$, $L$) traits to evaluate their impacts on canopy productivity. Plot *net* canopy photosynthesis ($A_{n,\mathrm{can}}$) to compare the changes in these parameters and discuss the results.
- Here we are integrating leaf photosynthesis spatially over a canopy but not temporally over a day or over a growing season. Hence the results represent 'instantaneous' canopy photosynthesis at one time point. Please discuss how this method may be incorporated into the growth functions in the form of $dW/dt = rW$ for temporal integration where $r$ is the intrinsic growth rate with respect to  carbon budget determining biomass that we have been discussing in previous units including unit 05. How would you do that? No coding is necessary for this sub-question.

### Breaking down the model into components: Irradiance

`Irradiance` system contains code related to the calculation of irradiance. Note that `Ih` and `I` are declared as `call` variable which can be *called* like a regular function. It is used when a certain equation needs to be evaluated multiple times with different values within the same time step. **Positional arguments** declared *before* semi-colon (`;`) indicates the list of depending variables and **keyword arguments** declared *after* semi-colon denotes input arguments that has to be supplied when the variable is called. For example, `Ih` depends on `I0` and `k` variables and accepts an argument named `l`. In the declaration of `I`, `Ih` is called with an argument named `l` which is supplied inside parenthesis following the name of variable (`Ih(l)`).

- Eqn 6.13 & 14

$$
\begin{align}
I_h(l) &= I_0 e^{-k l} \\
I(l) &= k \cdot I_h(l) = k \cdot I_0 e^{-k l}
\end{align}
$$

- Table 6.1

| Symbol | Units | Description |
|:-------|:------|:------------|
| $l$ | $\mathrm{m_{leaf}^2}\ \mathrm{m_{ground}^{-2}}$ | Cumulative leaf area index up to the position of a leaf inside the canopy |
| $I_h(l)$ | $$\mathrm{\mu mol}\ \mathrm{m^{-2}}\ \mathrm{s^{-1}}$$ | $I$ incident on a horizontal surface at canopy depth $l$ |
| $I(l)$ | $\mathrm{\mu mol}\ \mathrm{m^{-2}}\ \mathrm{s^{-1}}$ | $I$ incident on a leaf surface at canopy depth $l$ |

- Table 6.1

| Symbol | Value | Units | Description |
|:-------|:------|:------|:------------|
| $I_0$ | 575 | $$\mathrm{\mu mol}\ \mathrm{m^{-2}}\ \mathrm{s^{-1}}$$ | Irradiance at the top of the canopy |
| $k$ | 0.8 | $\mathrm{m_{ground}^2}\ \mathrm{m_{leaf}^{-2}}$ | Light extinction coefficient inside the canopy |

In [ ]:
@system Irradiance begin
    I0: irradiance_top   => 575 ~ preserve(parameter, u"μmol/m^2/s") # equivalent to 125 W m-2 of PAR
    k:  extinction_coeff => 0.8 ~ preserve(parameter, u"m^2/m^2")

    Ih(I0, k; l) => I0 * exp(-k*l) ~ call(u"μmol/m^2/s")
    I(Ih, k; l)  => k * Ih(l)      ~ call(u"μmol/m^2/s")
end

### Breaking down the model into components: $A_{can}$

`CanopyPhotosynthesis` consists of variables calculated by integrating over a depth of the canopy. Gross photosynthesis (`Ag`) is a `hold` variable implying an actual definition of this variable must be supplied by other systems (mix-ins) which will be later combined to form a complete system. Dark respiration (`Rd`) is calculated by the ratio parameter (`Rdp`) whose initial value is 0.1 assuming 10% of maximum gross photosynthesis (`Amax*L`) is used for canopy respiration (`Rd`).

$$R_d = 0.1 \cdot A_{\mathrm{max}} L$$

- Table 6.1

| Symbol | Units | Description |
|:-------|:------|:------------|
| $L$ | $\mathrm{m_{leaf}^2}\ \mathrm{m_{ground}^{-2}}$ | Total leaf area index of the canopy |
| $R_d$ | $$\mathrm{\mu mol}\ \mathrm{m^{-2}}\ \mathrm{s^{-1}}$$ | Canopy respiration rate |

- Table 6.1

| Symbol | Value | Units | Description |
|:-------|:------|:------|:------------|
| $\alpha$ | 0.0494 | $$\mathrm{\mu mol_{CO_2}}\ \mathrm{\mu mol_{photon}^{-1}}$$ | Apparent photochemical efficiency (*a.k.a.* quantum yield) |
| $A_{\mathrm{max}}$ | 22.73 | $\mathrm{\mu mol}\ \mathrm{m^{-2}}\ \mathrm{s^{-1}}$ | Light saturated $A_g$ at ambient $\mathrm{[CO_2]}$ and standard temperature |

In [ ]:
c = @config Photosynthesis => (Amax = 22.73, α = 0.0494)

In [ ]:
@system CanopyPhotosynthesis(Photosynthesis) begin
    α:    photochemical_efficiency    => 0.0494 ~ preserve(parameter, u"μmol/μmol")
    Amax: maximum_photosynthetic_rate => 22.73  ~ preserve(parameter, u"μmol/m^2/s")

    L: leaf_area_index => 5 ~ preserve(parameter, u"m^2/m^2") # for comparison with Gourdriaan and van Laar (1994)

    Rdp:              dark_respiration_ratio => 0.1        ~ preserve(parameter)
    Rd(Rdp, Amax, L): dark_respiration       => Rdp*Amax*L ~ track(u"μmol/m^2/s")
end

### Analytical solution

Let's define an anlytical version of the model first. `CanopyPhotosynthesisA` is composed of other systems including `CanopyPhotosynthesis` and `Irradiance` defined above. These mixins provide actual definition of many variables declared to be `hold` here. For example, `α`, `Amax`, and `Rd` `Ag` is provided by this system while other variables are declared to be `hold` meaning provided by other mixins.

- Eqn 6.16

$$
A_{g,\mathrm{can}} = \int_0^L A(l) \, dl = \frac{A_{\mathrm{max}}}{k} \cdot \ln{\frac{A_{\mathrm{max}} + \alpha k I_0}{A_{\mathrm{max}} + \alpha k I_0 e^{-k L}}}
$$

In [ ]:
@system CanopyPhotosynthesisA(CanopyPhotosynthesis, Irradiance, Controller) begin
    Ag(α, Amax, I0, k, L): gross_photosynthesis => begin
        Amax/k * log((Amax + α*k*I0) / (Amax + α*k*I0*exp(-k*L)))
    end ~ track(u"μmol/m^2/s")
end

In [ ]:
Cropbox.hierarchy(CanopyPhotosynthesisA)

`CanopyPhotosynthesisN` is an intermediate mix-in providing a common ground for Gauss and Euler version of the model. Instead of directly computing `Ag` for the whole canopy, it makes uses a `call` variable `A` for calculating a photosynthesis rate at a certain layer represented by a value of leaf area index (`L`). `A` will be later integrated over a total range of leaf area index from 0 to `L` for computing `Ag`.

- Eqn 6.15

$$
A(l) = \frac{\alpha I(l) \cdot A_{\mathrm{max}}}{\alpha I(l) + A_{\mathrm{max}}}
$$

- Table 6.1

| Symbol | Units | Description |
|:-------|:------|:------------|
| $A(l)$ | $$\mathrm{\mu mol}\ \mathrm{m^{-2}}\ \mathrm{s^{-1}}$$ | Leaf gross $\mathrm{CO_2}$ assimilation rate ($A_g$) of a leaf inside the canopy at depth $l$ |

In [ ]:
@system CanopyPhotosynthesisN(CanopyPhotosynthesis, Irradiance, Controller) begin
    A(α, Amax, I; l) => begin
        Il = I(l)
        α*Il * Amax / (α*Il + Amax)
    end ~ call(u"μmol/m^2/s")
end

### Gaussian integration

`CanopyPhotosynthesisG` uses Gaussian integration method to compute `Ag` from `A` defined above. Let's use QuadGK for convenience.

In [ ]:
@system CanopyPhotosynthesisG(CanopyPhotosynthesisN, Controller) begin
    Ag(A, L): gross_photosynthesis => begin
        #Cropbox.QuadGK.quadgk(A, 0, L) |> first
        X, W = Cropbox.QuadGK.gauss(3, 0, L)
        sum(W .* A.(X))
    end ~ track(u"μmol/m^2/s")
end

In [ ]:
Cropbox.hierarchy(CanopyPhotosynthesisG)

### Rectangular integration (Eulerian)

`CanopyPhotosynthesisE` uses Euler integration method to compute `Ag`. Overall structure is close to `CanopyPhotosynthesisG` except that it uses a parameter `dl` to control the size of integration step.

In [ ]:
@system CanopyPhotosynthesisE(CanopyPhotosynthesisN, Controller) begin
    dl => 1.0 ~ preserve(parameter)
    Ag(A, L, dl): gross_photosynthesis => begin
        sum([A(l) * dl for l in dl:dl:L])
    end ~ track(u"μmol/m^2/s")
end

In [ ]:
Cropbox.hierarchy(CanopyPhotosynthesisE)

Now it's time to make a plot of three models using the same option. We want to plot gross photosynthesis (`Ag`) against a range of leaf area index (`L`) from 0 to 15. `visualize()` function again conveniently provides a feature to draw a series of simulation lines from multiple systems.

Reproduce the results for three methods and compare them with the values from Gourdriaan and van Laar (1994) p. 181. Their analytical solution was 843.74 $\mathrm{\mu g}\ \mathrm{m^{-2}}\ \mathrm{s^{-1}}$ and Gaussian solution was 844.22 $\mathrm{\mu g}\ \mathrm{m^{-2}}\ \mathrm{s^{-1}}$ respectively. Note that 1 $\mathrm{\mu mol_{CO_2}}$ equals to 44 $\mathrm{\mu g_{CO_2}}$.

In [ ]:
co2_molar_mass = 44u"μg" / 1u"μmol"

In [ ]:
instance(CanopyPhotosynthesisA).Ag' * co2_molar_mass

In [ ]:
instance(CanopyPhotosynthesisG).Ag' * co2_molar_mass

In [ ]:
instance(CanopyPhotosynthesisE).Ag' * co2_molar_mass

Let's try a smaller integration step for the rectangular method. $dl$ is now decreased to 0.1 from 1.0 above.

In [ ]:
instance(CanopyPhotosynthesisE, config = :0 => :dl => 0.1).Ag' * co2_molar_mass

In [ ]:
models = [CanopyPhotosynthesisA, CanopyPhotosynthesisG, CanopyPhotosynthesisE]
xstep = :0 => :L => 0:0.1:10
names = ["Analytic", "Gauss", "Euler"]
kind = :scatter
ylim = (0, 30);

Note that the line of Gauss model (yellow) is barely visible since it's mostly overlapped with the line of analytic solution (blue).

In [ ]:
visualize(models, :L, :Ag; xstep, names, kind, ylim)

Plotting net photosynthesis (`An`) which takes out 10% of gross respiration (`Ag`) as dark respiration (`Rd`) can be easily done in a similar way.

In [ ]:
visualize(models, :L, :An; xstep, names, kind, ylim)

Note that Cropbox also implements a special kind of variable named `integrate` to cover the need of numerical integration as we explored in this notebook. Instead of relying on two separate variables, one `call` defining a function to be integrated and another `track` manually doing numerical integration, `integrate` can describe the same logic using a single variable. It takes a similar form as `call` putting the name of internal variable to be integrated (`l` in this case) after semi-colon (`;`) in the argument list. Then the range of integration is specified by `from` and `to` tags defined inside the parenthesis following the state name (`integrate`). As we want to integrate from 0 to `L` and the default value of `from` and `to` are 0, we only need to specify `to` as `L`. As a bonus, unit conversion would be automatically handled if any of these variables were assigned some units. `CanopyPhotosynthesisG2` implemented with `call` should behave in the exactly same way as `CanopyPhotosynthesisG` implemented earlier with `call` and `track`.

In [ ]:
@system CanopyPhotosynthesisG2(CanopyPhotosynthesis, Irradiance, Controller) begin
    Ag(α, Amax, I; l): gross_photosynthesis => begin
        Il = I(l)
        α*Il * Amax / (α*Il + Amax)
    end ~ integrate(to = L, u"μmol/m^2/s")
end

In [ ]:
Cropbox.hierarchy(CanopyPhotosynthesisG2)

In [ ]:
Cropbox.dependency(CanopyPhotosynthesisG2)

In [ ]:
instance(CanopyPhotosynthesisG2)

### Sensitivity of Parameters

We can test sensitivty of parameters as we did earlier.

In [ ]:
visualize(CanopyPhotosynthesisG2, :Amax, :An;
    xstep = :0 => :Amax => 1:50,
    kind = :line,
)

In [ ]:
visualize(CanopyPhotosynthesisG2, :α, :An;
    xstep = :0 => :α => 0:0.001:0.1,
    xlim = (0, 0.1),
    kind = :line,
)

In [ ]:
visualize(CanopyPhotosynthesisG2, :k, :An;
    xstep = :0 => :k => 0:0.01:1,
    kind = :line,
)

In [ ]:
visualize(CanopyPhotosynthesisG2, :L, :An;
    xstep = :0 => :L => 0:0.1:10,
    kind = :line,
)

#### Light response of leaf and canopy photosynthesis

In [ ]:
p = visualize(CanopyPhotosynthesisG2, :I0, :An;
    config = :0 => :L => 5,
    xstep = :0 => :I0 => 0:10:2000,
    kind = :line,
    name = "An,can (L=5)",
)
visualize!(p, PhotosynthesisRH, :I, :An;
    xstep = :0 => :I => 0:10:2000,
    kind = :line,
    name = "An,leaf",
)

We can make an interactive plot using `manipulate()` function.

In [ ]:
Cropbox.Interact.WebIO.setup(:ijulia)

In [ ]:
manipulate(CanopyPhotosynthesisG2, :I0, :An;
    parameters = :0 => :L => 0:0.1:10,
    xstep = :0 => :I0 => 0:10:2000,
    ylim = (-10, 30),
    kind = :line,
)

In [ ]:
manipulate(parameters = :0 => :L => 0:0.1:10) do c
    p = visualize(CanopyPhotosynthesisG2, :I0, :An;
        config = c,
        xstep = :0 => :I0 => 0:10:2000,
        ylim = (-10, 30),
        kind = :line,
        name = "An,can",
    )
    visualize!(p, PhotosynthesisRH, :I, :An;
        xstep = :0 => :I => 0:10:2000,
        kind = :line,
        name = "An,leaf",
    )
end

## Homework Problem 6.1

1. Using the Gaussian 3 point integration method, experiment with different parameter values for physiological (e.g., $A_{\mathrm{max}}$, $\alpha$, $\theta$) and canopy (e.g., $k$, $L$, $I_0$) traits to evaluate their impacts on canopy productivity; that is, perform sensitivity analyses of the parameters within a reasonable range of their expected values. $\textbf{(1)}$ Plot $\textit{net}$ canopy photosynthesis ($A_{n,can}$) in response to different values of the parameters and $\textbf{(2)}$ Interpret and discuss the results. $\textit{Hint:}$ Use 
    $\texttt{xstep}$ and $\texttt{group}$ for visualization.
2. Here we are integrating leaf photosynthesis spatially over the leaf area index of a canopy in steady state. Hence the results represent 'instantaneous' canopy photosynthesis at one time point. This gives a snapshot but is not a dynamic model that runs over a day or throughout the growing season. Discuss how this steady-state snapshot may be incorporated into a growth function in the form of $dW/dt = rW$ we've looked at before for temporal integration where $r$ is the specific growth rate. How would you do that? No coding is necessary for this part. Just discuss. 